In [145]:
import chardet 
import pandas as pd
import numpy as np

with open("train.csv", 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

print(result['encoding'])
df_raw = pd.read_csv('train.csv', encoding=result['encoding'], delimiter=',')



ascii


In [2]:
df_raw.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
from sklearn.model_selection import train_test_split

In [146]:

X = df_raw.drop(columns=['Survived'])
y = df_raw['Survived']

print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

(891, 11)


In [5]:
print("Shape of X is {}".format(X.shape))
print("Shape of X_train is {} and shape of y_train is {}".format(X_train.shape, y_train.shape))
print("Shape of X_test is {} and shape of y_test is {}".format(X_test.shape, y_test.shape))

train_corpus = list(X_train)

Shape of X is (891, 11)
Shape of X_train is (668, 11) and shape of y_train is (668,)
Shape of X_test is (223, 11) and shape of y_test is (223,)


In [6]:
X_train.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,668.000000,668.000000,541.000000,668.000000,668.000000,668.000000
mean,448.967066,2.305389,29.528189,0.535928,0.413174,33.740412
std,258.757008,0.833789,14.329970,1.164272,0.848284,52.836576
min,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,226.500000,2.000000,21.000000,0.000000,0.000000,7.925000
50%,458.500000,3.000000,28.000000,0.000000,0.000000,14.500000
75%,674.250000,3.000000,38.000000,1.000000,1.000000,31.387500
max,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [148]:
X

array([[ 0.    ,  3.    ,  1.    , ...,  0.    ,  7.25  ,  0.    ],
       [ 1.    ,  1.    ,  0.    , ...,  0.    , 71.2833,  1.    ],
       [ 1.    ,  3.    ,  0.    , ...,  0.    ,  7.925 ,  0.    ],
       ...,
       [ 0.    ,  3.    ,  0.    , ...,  2.    , 23.45  ,  0.    ],
       [ 1.    ,  1.    ,  1.    , ...,  0.    , 30.    ,  1.    ],
       [ 0.    ,  3.    ,  1.    , ...,  0.    ,  7.75  ,  2.    ]])

In [11]:
g = X_train.groupby(['Pclass','Cabin'])

In [14]:
len(list(filter(lambda x : x > 0, df_raw['Age'])))

714

In [18]:
cabins = sorted(map(str,df_raw['Cabin']))

In [19]:
import bisect 

In [40]:
len(list(filter(lambda x : x is np.nan , df_raw['Cabin'])))

687

## Baseline 

Using random forests on pclass, sex, sibsp, parch, fare, embarked


In [142]:
from sklearn import preprocessing

In [130]:
def baselineVectorizer(df):
# leEmbarked.fit(df_raw['Embarked'])
    leSex = preprocessing.LabelEncoder()
    leEmbarked = preprocessing.LabelEncoder()
    leSex.fit(df['Sex'])
    leEmbarked.fit(df['Embarked'])
    df.fillna(inplace=True, value=0)
    df['Embarked'] = leEmbarked.transform(df['Embarked'])
#     df['Embarked'] = list(map(lambda x : 0 if x is 'S' else 1 if x is 'C' else 2, df['Embarked']))
    df['Sex'] = leSex.transform(df['Sex'])
    return df.drop(labels=['PassengerId', 'Name', 'Age', 'Ticket', 'Cabin'], axis=1)

In [147]:
print(X)
X = baselineVectorizer(df_raw)
print(X.shape)

     PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
5              6       3                                   Moran, Mr. James   
6              7       1                            McCarthy, Mr. Timothy J   
7              8       3                     Palsson, Master. Gosta Leonard   
8              9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9             10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10            11       3                    Sandstrom, Miss. Marguerite Rut   
11            12       1                           B

In [71]:
X_train_b1 = baselineVectorizer(X_train)
X_test_b1  = baselineVectorizer(X_test)


/home/bhishma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/bhishma/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


array([[ 3.    ,  1.    ,  0.    ,  0.    ,  7.125 ,  2.    ],
       [ 3.    ,  1.    ,  0.    ,  0.    ,  7.05  ,  2.    ],
       [ 3.    ,  1.    ,  0.    ,  0.    ,  7.8958,  2.    ],
       ...,
       [ 2.    ,  0.    ,  0.    ,  0.    , 10.5   ,  2.    ],
       [ 2.    ,  1.    ,  0.    ,  0.    , 13.    ,  2.    ],
       [ 1.    ,  0.    ,  0.    ,  0.    , 86.5   ,  2.    ]])

In [140]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42)

print(X.shape)

model.fit(X, y)

# from sklearn.metrics import accuracy_score, confusion_matrix

# y_test_predicted = model.predict(X_test_b1)

# print("The fraction of correctly classified samples is {}".format(accuracy_score(y_test, y_test_predicted)))

# pd.DataFrame(confusion_matrix(y_test_predicted, y_test), 
#              index={'true s', 'true ns'}, 
#              columns={'pred s', 'pred ns'})


(891, 7)


/home/bhishma/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [88]:
from sklearn.model_selection import cross_val_score

lr_model = LogisticRegression()
cv_scores = cross_val_score(lr_model, X=baselineVectorizer(df_raw), y=y, cv=5, n_jobs=4)
print(cv_scores)

[0.80446927 0.80446927 0.78089888 0.76966292 0.81355932]


In [81]:
baselineVectorizer(df_raw).shape

(891, 7)

In [135]:
df_raw_test = pd.read_csv('test.csv')

In [136]:
df_test = baselineVectorizer(df_raw_test)

In [141]:
y_test_predicted = model.predict(df_test)

ValueError: X has 6 features per sample; expecting 7